In [1]:
import os
os.chdir("VASP_FILES")

# Running a DFT Calculation in VASP

The process begins by obtaining a Crystallographic Information File (CIF). The best website for this is: https://www.ccdc.cam.ac.uk/structures/?

## Input Files

In [2]:
%%bash
cat KBr.cif

# generated using pymatgen
data_KBr
_symmetry_space_group_name_H-M   Fm-3m
_cell_length_a   6.70307650
_cell_length_b   6.70307650
_cell_length_c   6.70307650
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   225
_chemical_formula_structural   KBr
_chemical_formula_sum   'K4 Br4'
_cell_volume   301.17750253
_cell_formula_units_Z   4
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
  2  '-x, -y, -z'
  3  'z, y, -x'
  4  '-z, -y, x'
  5  '-x, y, -z'
  6  'x, -y, z'
  7  '-z, y, x'
  8  'z, -y, -x'
  9  'x, -y, -z'
  10  '-x, y, z'
  11  'z, -y, x'
  12  '-z, y, -x'
  13  '-x, -y, z'
  14  'x, y, -z'
  15  '-z, -y, -x'
  16  'z, y, x'
  17  'y, -z, -x'
  18  '-y, z, x'
  19  'y, x, -z'
  20  '-y, -x, z'
  21  'y, z, x'
  22  '-y, -z, -x'
  23  'y, -x, z'
  24  '-y, x, -z'
  25  '-y, z, -x'
  26  'y, -z, x'
  27  '-y, -x, -z'
  28  'y, x, z'
  29  '-y, -z, x'
  30  'y, z, -x'
  31  '-y, 

Using the command:
    
cif2cell Name.cif -p vasp --setup-all

will produce 4 files: INCAR, KPOINTS, POSCAR and POTCAR

In [3]:
%%bash
cat INCAR_C2C

# Generated by cif2cell 1.2.11.
ENCUT = 324.4275
#NBANDS = 27
PREC = Accurate
LREAL = Auto
ISMEAR = 0
SIGMA = 0.1

The INCAR file determines the parameters of the calculation.

In [4]:
%%bash
cat KPOINTS

Gamma-point only
0
Monkhorst Pack
9 9 9
0 0 0


The KPOINT file determines where the calculation is centred, which for our purposes is always the gamma point. It also determines the number of sample points where the wavefunction is determined.

In [5]:
%%bash
cat POSCAR

Generated by cif2cell 1.2.11.  :  Failed to get author information, No journal information. Species order: K Br 
   6.703076
  0.000000000000000   0.500000000000000   0.500000000000000
  0.500000000000000   0.000000000000000   0.500000000000000
  0.500000000000000   0.500000000000000   0.000000000000000
   1   1
Direct
  0.000000000000000   0.000000000000000   0.000000000000000 
  0.500000000000000   0.500000000000000   0.500000000000000 


The POSCAR file contains the starting positions of the ions for the calculation in the VASP format, extracted from the CIF file.

In [6]:
%%bash
cat POTCAR

  PAW_PBE K_pv 17Jan2003                 
   7.00000000000000     
 parameters from PSCTR are:
   VRHFIN =K:  p6s1
   LEXCH  = PE
   EATOM  =   450.4813 eV,   33.1094 Ry

   TITEL  = PAW_PBE K_pv 17Jan2003
   LULTRA =        F    use ultrasoft PP ?
   IUNSCR =        1    unscreen: 0-lin 1-nonlin 2-no
   RPACOR =    2.500    partial core radius
   POMASS =   39.098; ZVAL   =    7.000    mass and valenz
   RCORE  =    3.100    outmost cutoff radius
   RWIGS  =    4.300; RWIGS  =    2.275    wigner-seitz radius (au A)
   ENMAX  =  116.731; ENMIN  =   87.548 eV
   RCLOC  =    2.806    cutoff for local pot
   LCOR   =        T    correct aug charges
   LPAW   =        T    paw PP
   EAUG   =  291.424
   DEXC   =    0.000
   RMAX   =    3.171    core radius for proj-oper
   RAUG   =    1.300    factor for augmentation sphere
   RDEP   =    3.267    radius for radial grids
   RDEPT  =    2.513    core radius for aug-charge
 
   Atomic configuration
    7 entries
     n  l   j            E   

The POTCAR file contains the information to construct the appropriate pseudopotential, which is how the nucleus and core electrons are modelled. If creating this file, the order of the atoms in the POSCAR must match the order in the POTCAR. Once created, this should not be edited for all subsequent calculations.

### Starting A VASP Calculation

To start a VASP calculation, call the VASP executable in the directory containing the files shown above.

For SPE calculations, 1-4 cores is typically enough to easily finish within 48 hours.

For all other calculations 24 or 48 cores are typically used, although more can be used for larger systems if necassary.

It should be noted that the suffixes used in this notebook are simply for file management. Each calculation <b>requires</b> that the files be labelled INCAR, POSCAR, POTCAR and KPOINTS. 

The output file will always be labelled OUTCAR and the wavefunction is written to the WAVECAR file. The WAVECAR file is important for continuing calculations when they haven't completed in the number of steps (NSW) or the allowed time. 

## Convergence

To ensure high accuracy, which is very important for phonon calculations, the parameters must be converged. The two parameters in VASP to be converged are KPOINTS and ENCUT. 

This is done with single-point (SPE) energy calculations in order to save time and resources.

In [7]:
%%bash
cat INCAR_SPE

ENCUT = 600     # Cut off Energy
PREC = Accurate # Sets precision - related to ENCUT
LREAL = .FALSE. # Real or reciprocal space
ISMEAR = -5     # Sets orbital partial occupancy
SIGMA = 0.05    # Sets width in eV of smearing
IBRION = -1      # Sets method of moving ions.
POTIM = 0.2     # Step widths
ISIF = 3        # Optimise ions, cell dimensions or both
NSW = 0         # Number of iterations before finishing job.
NELMIN = 4      # Min number of SCF cycles for each set of ionic positions
ALGO = Fast     # Determines what algorithm is used
EDIFF = 5.0E-6  # Determines accuracy at which program stops
LMAXMIX = 4     # This and the following 4 parameters are optional to improve efficiency
NFREE = 10
LPLANE = .TRUE.
NCORE = 8
NSIM = 2


This INCAR file is used to set up a SPE calculation. The tags that have a comment after them are necessary, while those without are optional and control the efficiency of the calculation.

Whilst varying either KPOINTS or ENCUT, the other value should remain fixed. The ENCUT and KPOINTS provided by cif2cell are mostly suitable for this.

When the calculation is finished, a useful command is:

In [8]:
%%bash
grep 'free  e' OUTCAR_SPE

  free  energy   TOTEN  =      -536.93366933 eV


This will output the calculated energy for that set of atomic positions. As we are doing an SPE, there will only be one value. By looking at the difference between this value and the one for another INCAR, parameter convergence can be determined.

There is no one defining metric for when a system is converged, but examining the change of the $4^{th}$ significant figure (SF) is generally a suitable place to start. 

## Geometry Optimisation

Once suitable values for KPOINTS and ENCUT have been chosen, make a new directory with a new set of VASP files. The POTCAR and POSCAR will remain unchanged, whilst the KPOINT file should contain your selected number of KPOINTS.

The INCAR for a geometry optimisation (GO) is as follows:

In [9]:
%%bash
cat INCAR_GO

ENCUT = 600     # Cut off Energy
PREC = Accurate # Sets precision - related to ENCUT
LREAL = .FALSE. # Real or reciprocal space
ISMEAR = -5     # Sets orbital partial occupancy
SIGMA = 0.05    # Sets width in eV of smearing
IBRION = 1      # Sets method of moving ions.
POTIM = 0.2     # Step widths
ISIF = 3        # Optimise ions, cell dimensions or both
NSW = 50        # Number of iterations before finishing job
NELMIN = 4      # Min number of SCF cycles for each set of ionic positions
ALGO = Fast     # Determines what algorithm is used
EDIFF = 5.0E-6  # Determines accuracy at which program stops
LMAXMIX = 4     # This and the following 4 parameters are optional to improve optimisation
NFREE = 10
LPLANE = .TRUE.
NCORE = 8
NSIM = 2


The INCAR is the same as for an SPE calculation with the exception of the IBRION and NSW tags. NSW is set to 50 here, but can be set to higher values if your system is not constrained to time-slots.

When the calculation has finished, the following command will reveal whether the GO had successfully completed:

In [10]:
%%bash
grep accuracy OUTCAR_GO

 reached required accuracy - stopping structural energy minimisation


If no output is given, the calculation must be continued. This is done by making a new directory and copying the CONTCAR, INCAR, KPOINTS, POTCAR and WAVECAR over. 

The CONTCAR should be renamed to POSCAR, and two tags should be added to the INCAR:

ISTART = 1 and ICHARG = 0

These will make the calculation read the WAVECAR and start where the last calculation finished.

## Fixed Cell Geometry Optimisation

To ensure the high degree of accuracy required for calculating phonon mode frequecies, a further GO is performed but without varying the cell dimensions. This ensures that the atomic positions are optimised within the unit cell itself which can drastically affect the effective forces. 

Copy the CONTCAR, KPOINTS, POTCAR and WAVECAR to a new directory and create a new INCAR as below:

In [11]:
%%bash
cat INCAR_FCGO

ENCUT = 600
PREC   = Accurate
LREAL  = .FALSE.
ISMEAR = -5      
SIGMA  = 0.05     
IBRION = 1       
POTIM  = 0.4     
ISIF   = 2       
NSW    = 50      
NELMIN = 4       
ALGO   = Fast
EDIFF  = 1.0E-7
EDIFFG = -5.0E-4
LMAXMIX = 4
NFREE   = 10
LPLANE = .TRUE.
NCORE   = 8
NSIM    = 2
ISTART =  1
ICHARGE = 0

The main difference is ISIF = 2 and EDIFF = 1.0E-7. This fixes the cell dimensions and performs the GO at tighter criteria.

Whether the calculation has completed or not can be deduced using the same command as above. If it hasn't, the restarting process is the same as above.

## Calculation of Phonon Frequencies

Once an optimised structure has been obtained, the dielectric tensor can be calculated using Density Functionl Pertubation Theory (DFPT).

This allows the calculation of the Born effective charges, from which the phonon mode frequencies are calculated.

As before, the output and configuration files should be copied to a new directory and renamed appropriately. The INCAR for this calculation is as follows:

In [12]:
%%bash
cat INCAR_D

ENCUT = 600
PREC   = Accurate
LREAL  = .FALSE.
ISMEAR = -5      
SIGMA  = 0.05     
IBRION = 8        
POTIM  = 0.2   
ISIF   = 3     
NSW    = 1     
NELMIN = 4     
EDIFF  = 1.0E-8   
LEPSILON = .TRUE. ! calculate dielectric constants
LRPA = .FALSE.
LMAXMIX = 4
ISTART =  1
ICHARGE = 0

Once this calculation has finished, use the following command to check whether it was successful:

In [13]:
%%bash
grep THz OUTCAR_D

   1 f  =    3.032958 THz    19.056636 2PiTHz  101.168582 cm-1    12.543310 meV
   2 f  =    3.032958 THz    19.056636 2PiTHz  101.168582 cm-1    12.543310 meV
   3 f  =    3.032958 THz    19.056636 2PiTHz  101.168582 cm-1    12.543310 meV
   4 f/i=    0.008513 THz     0.053491 2PiTHz    0.283973 cm-1     0.035208 meV
   5 f/i=    0.008513 THz     0.053491 2PiTHz    0.283973 cm-1     0.035208 meV
   6 f/i=    0.008513 THz     0.053491 2PiTHz    0.283973 cm-1     0.035208 meV


If nothing appears, the calculation did not finish in the allowed time (this only applies to people using time-limited systems), and another program, Phonopy, must be used. 

If the calculation was successful, the output can be analysed further in PDielec.